In [1]:
# 载入此项目所需要的库
import numpy as np
import pandas as pd

In [2]:
#载入北京房价数据
data = pd.read_csv('bj_housing.csv')
# print data
price_data = data['Value']
features_all_data = data.drop(['Value'],axis = 1)
# print features_all_data


In [3]:
#清洗数据
#先去掉年份小于1900的，这个数据分明是错误的


In [4]:
#  分割数据
from sklearn.cross_validation import train_test_split
features_train, features_test, price_train, price_test = train_test_split(features_all_data, price_data, test_size=0.2, random_state=5)

D:\Anaconda2\envs\py2_env\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [5]:
#用来预测分值的函数
def performance_metric(y_true, y_predict):
    """计算并返回预测值相比于预测值的分数"""
    from sklearn.metrics import r2_score
    score = r2_score(y_true, y_predict)

    return score

In [6]:
#创建GridSearchCV来查找最好的决策树深度模型
#提示: 导入 'KFold' 'DecisionTreeRegressor' 'make_scorer' 'GridSearchCV' 
from sklearn.cross_validation import KFold
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.model_selection import GridSearchCV

def fit_DecisionTreeRegressor_model(X, y):
    """ 基于输入数据 [X,y]，利于网格搜索找到最优的决策树模型"""
    
    cross_validator =  KFold(len(y), shuffle=True)
    
    regressor = DecisionTreeRegressor()

    params =  {'max_depth':[1,2,3,5,6,7,8,9,10]}

    scoring_fnc = make_scorer(performance_metric)

    grid = GridSearchCV(regressor, params, scoring_fnc, cv = cross_validator)

    # 基于输入数据 [X,y]，进行网格搜索
    grid = grid.fit(X, y)

    # 返回网格搜索后的最优模型
    return grid.best_estimator_

In [7]:
# 基于训练数据，获得最优模型
#决策树回归模型秒找到
optimal_reg = fit_DecisionTreeRegressor_model(features_train, price_train)
print "回归模型的最优参数：",optimal_reg.get_params()

回归模型的最优参数： {'presort': False, 'splitter': 'best', 'min_impurity_decrease': 0.0, 'max_leaf_nodes': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'criterion': 'mse', 'random_state': None, 'min_impurity_split': None, 'max_features': None, 'max_depth': 5}


In [8]:
#创建GridSearchCV来查找最好的SVR模型
#提示: 导入 'KFold' 'DecisionTreeRegressor' 'make_scorer' 'GridSearchCV' 
from sklearn.cross_validation import KFold
from sklearn.svm import SVR
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.model_selection import GridSearchCV

def fit_SVR_model(X, y):
    """ 基于输入数据 [X,y]，利于网格搜索找到最优的决策树模型"""
    
    cross_validator =  KFold(len(y), shuffle=True)
    
    regressor = SVR(kernel='rbf',epsilon=0.2)

    params =  {'C': [1e3, 5e3, 1e4, 5e4, 1e5],
          'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1],}

    scoring_fnc = make_scorer(performance_metric)

    grid = GridSearchCV(regressor, params, scoring_fnc, cv = cross_validator)

    # 基于输入数据 [X,y]，进行网格搜索
    grid = grid.fit(X, y)

    # 返回网格搜索后的最优模型
    return grid.best_estimator_

In [49]:
# 基于训练数据，获得最优模型
#决策树回归模型秒找到
#SVR将近花了一个小时才找到
optimal_reg = fit_SVR_model(features_train, price_train)
print "回归模型的最优参数：",optimal_reg.get_params()

回归模型的最优参数： {'kernel': 'rbf', 'C': 5000.0, 'verbose': False, 'degree': 3, 'epsilon': 0.2, 'shrinking': True, 'max_iter': -1, 'tol': 0.001, 'cache_size': 200, 'coef0': 0.0, 'gamma': 0.001}


In [9]:
#创建GridSearchCV来查找最好的神经网络模型
#提示: 导入 'KFold' 'DecisionTreeRegressor' 'make_scorer' 'GridSearchCV' 
from sklearn.cross_validation import KFold
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler  

from sklearn.neural_network import MLPRegressor

scaler = StandardScaler()
def fit_SVR_model(X, y):
    """ 基于输入数据 [X,y]，利于网格搜索找到最优的神经网络模型"""
    
    #这个模型对于数据缩放比较敏感
    scaler.fit(X)
    #进行缩放
    X = scaler.transform(X)
    
    cross_validator =  KFold(len(y), shuffle=True)
    
    regressor = MLPRegressor(solver='lbfgs', hidden_layer_sizes=(20, 20, 20), random_state=1)
    
    regressor.fit(X,y)
    
    return regressor

#     params =  {'C': [1e3, 5e3, 1e4, 5e4, 1e5],
#           'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1],}

#     scoring_fnc = make_scorer(performance_metric)

#     grid = GridSearchCV(regressor, params, scoring_fnc, cv = cross_validator)

#     # 基于输入数据 [X,y]，进行网格搜索
#     grid = grid.fit(X, y)

#     # 返回网格搜索后的最优模型
#     return grid.best_estimator_

In [10]:

optimal_reg = fit_SVR_model(features_train, price_train)
print "回归模型的最优参数：",optimal_reg.get_params()

回归模型的最优参数： {'beta_1': 0.9, 'warm_start': False, 'beta_2': 0.999, 'shuffle': True, 'verbose': False, 'nesterovs_momentum': True, 'hidden_layer_sizes': (20, 20, 20), 'epsilon': 1e-08, 'activation': 'relu', 'max_iter': 200, 'batch_size': 'auto', 'power_t': 0.5, 'random_state': 1, 'learning_rate_init': 0.001, 'tol': 0.0001, 'validation_fraction': 0.1, 'alpha': 0.0001, 'solver': 'lbfgs', 'momentum': 0.9, 'learning_rate': 'constant', 'early_stopping': False}


In [11]:
scale_test_features = scaler.transform(features_test)

#查看预测分值
#下面两种方式算出来的值一样
# r2 = optimal_reg.score(X_test,y_test)
predict_res = optimal_reg.predict(scale_test_features)
r2 = performance_metric(price_test,predict_res)

print "R平方分值 {:,.2f} on test data".format(r2)

R平方分值 -0.30 on test data
